#### Run relevance backout here

In [1]:
import pickle
import re
import os

import random
import numpy as np
import torch
from random import shuffle
import argparse
import pickle

import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import sys
sys.path.append("..")

from model.BERT import *

from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from tqdm import tqdm, trange

from util.optimization import BERTAdam
from util.processor import *

from util.tokenization import *

from util.evaluation import *

import logging
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s', 
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

from sklearn.metrics import classification_report

# this imports most of the helpers needed to eval the model
from run_classifier import *

lrp_data_dir = "../../results"
vocab_data_dir = "../../data/uncased_L-12_H-768_A-12/vocab.txt"
sys.path.append("..")
import operator

#### Set-ups

In [2]:
# Note that this notebook only supports single GPU evaluation
# which is sufficient for most of tasks by using lower batch size.
IS_CUDA = False
if IS_CUDA:
    CUDA_DEVICE = "cuda:5"
    device = torch.device(CUDA_DEVICE)
    n_gpu = torch.cuda.device_count()
    logger.info("device %s in total n_gpu %d distributed training", device, n_gpu)
else:
    # bad luck, we are on CPU now!
    logger.info("gpu is out of the picture, let us use CPU")
    device = torch.device("cpu")

10/30/2020 01:23:11 - INFO - run_classifier -   gpu is out of the picture, let us use CPU


#### Indicate your folders

In [3]:
TASK_NAME = "SST5"
DATA_DIR = "../../data/dataset/SST5/"
            
# "../../data/uncased_L-12_H-768_A-12/" is for the default BERT-base pretrain
BERT_PATH = "../../data/uncased_L-12_H-768_A-12/"
MODEL_PATH = "../../results/" + TASK_NAME + "/checkpoint.bin"
EVAL_BATCH_SIZE = 24 # you can tune this down depends on GPU you have.

# This loads the task processor for you.
processors = {
    "IMDb":IMDb_Processor,
    "SemEval":SemEval_Processor,
    "SST5":SST5_Processor,
    "SST2":SST2_Processor,
    "SST3":SST3_Processor,
    "Yelp5":Yelp5_Processor,
    "Yelp2":Yelp2_Processor,
    "AdvSA":AdvSA_Processor
}

processor = processors[TASK_NAME]()
label_list = processor.get_labels()

In [4]:
model, optimizer, tokenizer = \
    getModelOptimizerTokenizer(model_type="BERTPretrain",
                               vocab_file=BERT_PATH + "vocab.txt",
                               embed_file=None,
                               bert_config_file=BERT_PATH + "bert_config.json",
                               init_checkpoint=MODEL_PATH,
                               label_list=label_list,
                               do_lower_case=True,
                               # below is not required for eval
                               num_train_steps=20,
                               learning_rate=2e-5,
                               base_learning_rate=2e-5,
                               warmup_proportion=0.1,
                               init_lrp=True)
model = model.to(device) # send the model to device

10/30/2020 01:23:11 - INFO - run_classifier -   model = BERTPretrain


init_weight = True
init_lrp = True


In [5]:
test_examples = processor.get_test_examples(DATA_DIR)
test_features = \
    convert_examples_to_features(
        test_examples,
        label_list,
        512,
        tokenizer)

all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in test_features], dtype=torch.long)
all_seq_len = torch.tensor([[f.seq_len] for f in test_features], dtype=torch.long)

test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                          all_label_ids, all_seq_len)
test_dataloader = DataLoader(test_data, batch_size=EVAL_BATCH_SIZE, shuffle=False)

 14%|█▍        | 315/2210 [00:00<00:00, 3137.66it/s]

0
guid= test-0
text_a= no movement , no yuks , not much of anything .
text_b= None
label= 1
1000
guid= test-1000
text_a= has all the poignancy of a hallmark card and all the comedy of a gallagher stand up act .
text_b= None
label= 2
2000
guid= test-2000
text_a= it 's still worth a look .
text_b= None
label= 3


100%|██████████| 2210/2210 [00:00<00:00, 2444.65it/s]


#### Call evaluation loop to get accuracy and attribution scores

In [6]:
# we did not exclude gradients, for attribution methods
model.eval() # this line will deactivate dropouts
test_loss, test_accuracy = 0, 0
nb_test_steps, nb_test_examples = 0, 0
pred_logits = []
actual = []

lrp_scores = []
inputs_ids = []
seqs_lens = []

# we don't need gradient in this case.
for step, batch in enumerate(tqdm(test_dataloader, desc="Iteration")):
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    input_ids, input_mask, segment_ids, label_ids, seq_lens = batch
    # truncate to save space and computing resource
    max_seq_lens = max(seq_lens)[0]
    input_ids = input_ids[:,:max_seq_lens]
    input_mask = input_mask[:,:max_seq_lens]
    segment_ids = segment_ids[:,:max_seq_lens]

    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)
    seq_lens = seq_lens.to(device)

    # intentially with gradient
    tmp_test_loss, logits = \
        model(input_ids, segment_ids, input_mask, seq_lens,
                device=device, labels=label_ids)

    # for lrp
    LRP_class = len(label_list) - 1
    Rout_mask = torch.zeros((input_ids.shape[0], len(label_list))).to(device)
    Rout_mask[:, LRP_class] = 1.0
    relevance_score = logits*Rout_mask
    lrp_score = model.backward_lrp(relevance_score).sum(dim=-1).cpu().data
    input_ids = input_ids.cpu().data
    seq_lens = seq_lens.cpu().data
    lrp_scores.append(lrp_score)
    inputs_ids.append(input_ids)
    seqs_lens.append(seq_lens)
    
    # for gradient
    
    # for attention only tracing
    
    
    logits = F.softmax(logits, dim=-1)
    logits = logits.detach().cpu().numpy()
    pred_logits.append(logits)
    label_ids = label_ids.to('cpu').numpy()
    actual.append(label_ids)
    outputs = np.argmax(logits, axis=1)
    tmp_test_accuracy=np.sum(outputs == label_ids)

    test_loss += tmp_test_loss.mean().item()
    test_accuracy += tmp_test_accuracy

    nb_test_examples += input_ids.size(0)
    nb_test_steps += 1
    
test_loss = test_loss / nb_test_steps
test_accuracy = test_accuracy / nb_test_examples

result = collections.OrderedDict()
result = {'test_loss': test_loss,
            str(len(label_list))+ '-class test_accuracy': test_accuracy}
logger.info("***** Eval results *****")
for key in result.keys():
    logger.info("  %s = %s\n", key, str(result[key]))
# get predictions needed for evaluation
pred_logits = np.concatenate(pred_logits, axis=0)
actual = np.concatenate(actual, axis=0)
pred_label = np.argmax(pred_logits, axis=-1)

lrp_state_dict = dict()
lrp_state_dict["lrp_scores"] = lrp_scores
lrp_state_dict["inputs_ids"] = inputs_ids
lrp_state_dict["seqs_lens"] = seqs_lens
logger.info("***** Finish LRP *****")

Iteration: 100%|██████████| 93/93 [13:47<00:00,  8.90s/it]
10/30/2020 01:43:27 - INFO - run_classifier -   ***** Eval results *****
10/30/2020 01:43:27 - INFO - run_classifier -     test_loss = 1.2114883481815297

10/30/2020 01:43:27 - INFO - run_classifier -     5-class test_accuracy = 0.5425339366515837



#### Aggregated lrp scores on a token aggregated across a dataset

In [7]:
def inverse_mapping(vocab_dict):
    inverse_vocab_dict = {}
    for k, v in vocab_dict.items():
        inverse_vocab_dict[v] = k
    return inverse_vocab_dict

def translate(token_ids, vocab):
    tokens = []
    for _id in token_ids.tolist():
        tokens.append(vocab[_id])
    return tokens

SST-5

In [13]:
vocab = inverse_mapping(load_vocab(vocab_data_dir, pretrain=False))
word_lrp = {}
word_lrp_list = []
for batch_idx in range(len(inputs_ids)):
    for seq_idx in range(inputs_ids[batch_idx].shape[0]):
        seq_len = seqs_lens[batch_idx][seq_idx].tolist()[0]
        tokens = translate(inputs_ids[batch_idx][seq_idx], vocab)[:seq_len]
        lrp_ss = lrp_scores[batch_idx][seq_idx].tolist()[:seq_len]
        for i in range(len(tokens)):
            word_lrp_list.append((tokens[i], lrp_ss[i]))
            if tokens[i] in word_lrp.keys():
                word_lrp[tokens[i]].append(lrp_ss[i])
            else:
                word_lrp[tokens[i]] = [lrp_ss[i]]
filter_word_lrp = {}
for k, v in word_lrp.items():
    if len(v) > 1:
        filter_word_lrp[k] = sum(v)*1.0/len(v)
filter_word_lrp = [(k, v) for k, v in filter_word_lrp.items()] 
filter_word_lrp.sort(key = lambda x: x[1], reverse=True)  
word_lrp_list.sort(key = lambda x: x[1], reverse=True)  

In [9]:
word_lrp_list[:20]

[('actress', 1.1658735275268555),
 ('movie', 1.0500624179840088),
 ('!', 1.0485429763793945),
 ('disturbing', 1.0441009998321533),
 ('fascinating', 1.0142135620117188),
 ('i', 0.9432353377342224),
 ('loved', 0.9145122170448303),
 ('excellent', 0.8970187306404114),
 ('wonderful', 0.8715908527374268),
 ('documentary', 0.857614278793335),
 ('thriller', 0.8538416624069214),
 ('[SEP]', 0.8367592096328735),
 ('comedy', 0.8343492746353149),
 ('accessible', 0.8268217444419861),
 ('tremendous', 0.8265071511268616),
 ('[CLS]', 0.8011797666549683),
 ('enjoy', 0.7982317209243774),
 ('hilarious', 0.7917709350585938),
 ('funny', 0.7913933396339417),
 ('delightful', 0.7900826334953308)]

In [10]:
word_lrp_list[-20:]

[('bother', -0.6187527179718018),
 ('caution', -0.6328242421150208),
 ('.', -0.6360901594161987),
 ('[SEP]', -0.6403815150260925),
 ('.', -0.6450625658035278),
 ('scenes', -0.6467615365982056),
 ('tale', -0.6688672304153442),
 ('[SEP]', -0.6737954020500183),
 ('why', -0.6740109920501709),
 ('.', -0.6845172643661499),
 ('weaker', -0.6860779523849487),
 ('[SEP]', -0.7098051905632019),
 ('[SEP]', -0.7335551977157593),
 ('[SEP]', -0.7473068237304688),
 ('search', -0.7814841270446777),
 ('fits', -0.7854188680648804),
 ('boring', -0.7887516021728516),
 ('hmm', -0.8135195970535278),
 ('sorry', -0.8380900621414185),
 ('confusing', -0.9827263355255127)]

In [14]:
filter_word_lrp[:20]

[('exceeding', 0.5255683809518814),
 ('glorious', 0.523917555809021),
 ('amazing', 0.48885713145136833),
 ('amazingly', 0.4737926498055458),
 ('marvelous', 0.4553905725479126),
 ('observed', 0.40511031200488407),
 ('masterpiece', 0.3976913373917341),
 ('visuals', 0.3964625895023346),
 ('brutal', 0.3908869996666908),
 ('beautifully', 0.3795804445232664),
 ('wonderful', 0.37883903458714485),
 ('playful', 0.37169771393140155),
 ('remarkable', 0.36964475611845654),
 ('tremendous', 0.36521638184785843),
 ('manners', 0.36161354929208755),
 ('loved', 0.35561820715665815),
 ('impress', 0.3487958957751592),
 ('decade', 0.34550681710243225),
 ('troubled', 0.33895863592624664),
 ('landmark', 0.3376755118370056)]

In [15]:
filter_word_lrp[-20:]

[('derivative', -0.2808782085776329),
 ('wander', -0.28310608863830566),
 ('search', -0.2873449847102165),
 ('exercise', -0.2924286097288132),
 ('exploitation', -0.2969381492584944),
 ('##asse', -0.301093265414238),
 ('lifeless', -0.3042931407690048),
 ('cheese', -0.309484027326107),
 ('pluto', -0.3161681070923805),
 ('blur', -0.31875964999198914),
 ('##typical', -0.33823950588703156),
 ('scotland', -0.3510182946920395),
 ('rabbits', -0.35597583651542664),
 ('bother', -0.36322491243481636),
 ('trivial', -0.3873773664236069),
 ('sinks', -0.4133940041065216),
 ('progress', -0.45691321790218353),
 ('fits', -0.48892583698034286),
 ('confusing', -0.5344277396798134),
 ('sorry', -0.6642181724309921)]